In [33]:
%pip install openai
%pip install PyPDF2


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.3 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [34]:
from openai import AzureOpenAI
import json
from PyPDF2 import PdfReader

In [35]:
COMPLETIONS_MODEL = "GPT4-1106"

client = AzureOpenAI(
    azure_endpoint="https://nuevayorkopenaiservice.openai.azure.com/",
    api_key="77c21ef531224c41b2581310f11e4aea",
    api_version="2023-12-01-preview",
)

In [41]:
# creating a pdf reader object 
reader = PdfReader('pdfs/ZEROeGLOBAL_ENAGÁS_20230718.pdf') 
  
nPaginas = len(reader.pages)
contenido = ""
  
for i in range(nPaginas):
    page = reader.pages[i]
    contenido += page.extract_text()
 
print(contenido)

info@zeroeglobal.comPresentación a ENAGÁS -18 de Julio 2023
ZEROeGLOBAL  Solutions
Producción amoniaco con procesos no Haber -Bosch - (Actualización) 18 de Julio 2023
ESTRICTAMENTE PRIVADO Y CONFIDENCIAL
12II - Producción amoniaco con procesos no Haber -Bosch
1.- OBJETIVO Y TECNOLOGÍAS
2.- ULTIMOS ACONTECIMIENTOS INTERNACIONALES 
3.-QUÉ ES Y QUIEN PRODUCE EL MATERIAL C12A7:e “electride”
4.-ASPECTOS DESTACABLES DEL MATERIAL C12A7:e “electride” PRODUCIDO EN ESPAÑA 
5.-ESTADO ACTUAL Y CONCLUSIONES
Presentación a ENAGÁS
info@zeroeglobal.com ESTRICTAMENTE PRIVADO Y CONFIDENCIALI - ZEROeGLOBAL  Solutions , S.L.
1.- MISIÓN y VISIÓN
2.- SOCIOS FUNDADORES
3.-ENFOQUE SECTORIAL18 de Julio de  2023info@zeroeglobal.com ESTRICTAMENTE PRIVADO Y CONFIDENCIAL18 de Julio de  2023
Ingeniería concurrente -    Integración    -    Patentes 
3Presentación a ENAGÁS
I -ZEROeGLOBAL Solutions, S.L.
1.- Misión
ZEROeGLOBAL  Solutions , S.L. se constituye el 13 mayo de 2022 para proporcionar 
soluciones cero emisio

In [24]:
def run_conversation():
    # Step 1: send the conversation and available functions to the model
    messages = [{"role": "system", "content": "Enagás es una empresa del sector energético"},
                {"role": "user", "content":"¿Cuál es el sector de Enagás?"}]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_evaluation",
                "description": "Get the evaluation",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "evaluation_score": {
                            "type": "integer",
                            "description": "Score",
                        },
                        "reasoning": {
                            "type": "string",
                            "description": "A reasoning of how the answer was created"},
                        },
                    "required": ["evaluation_score"],
                },
            },
        }
    ]

    response = client.chat.completions.create(
        model=COMPLETIONS_MODEL,
        messages=messages,
        tools=tools,
        tool_choice={"type": "function", "function": {"name": "get_evaluation"}},
    )
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    print(tool_calls)
    # Step 2: check if the model wanted to call a function
    if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        json_str = tool_calls[0].function.arguments
        dict_args = json.loads(json_str)
        evaluation_score = dict_args['evaluation_score']
        reasoning = dict_args['reasoning']
        print("Evaluation score: " + str(evaluation_score))
        print("Reasoning: " + reasoning)

    return response_message

In [25]:
print(run_conversation())

[ChatCompletionMessageToolCall(id='call_3FlWjNyotofsnBrrAFXZFuI1', function=Function(arguments='{"evaluation_score":10,"reasoning":"La empresa Enagás opera en el sector energ�tico, espec�ficamente en el sector del gas natural. Es una de las principales compa��as de transporte y gesti�n de infraestructuras de gas natural en Espa�a y tiene una participaci�n significativa en el desarrollo y operaci�n del sistema gasista espa�ol y en actividades internacionales relacionadas con el gas natural."}', name='get_evaluation'), type='function')]
Evaluation score: 10
Reasoning: La empresa Enagás opera en el sector energ�tico, espec�ficamente en el sector del gas natural. Es una de las principales compa��as de transporte y gesti�n de infraestructuras de gas natural en Espa�a y tiene una participaci�n significativa en el desarrollo y operaci�n del sistema gasista espa�ol y en actividades internacionales relacionadas con el gas natural.
ChatCompletionMessage(content=None, role='assistant', function_c